In this notebook, I will be using chunked retrieval i.e., chunking each document and then creating their embeddings for retrieval later. Only the most informative or similar chunks will be given as reference to the LLM.

In [3]:
import pandas as pd
import numpy as np
import faiss
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModel
from sentence_transformers import SentenceTransformer
import torch

In [ ]:
# Load Dataset
train = pd.read_csv("/teamspace/studios/this_studio/LegalTech-Palak/train.csv")
test = pd.read_csv("/teamspace/studios/this_studio/LegalTech-Palak/test.csv")

df = pd.concat([train, test])

In [ ]:
# Load InLegalBERT
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

embed_tokenizer = AutoTokenizer.from_pretrained("law-ai/InLegalBERT")
embed_model = AutoModel.from_pretrained("law-ai/InLegalBERT").to(device)
embed_model.eval()

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/516 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/671 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/534M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

! huggingface-cli download meta-llama/Meta-Llama-3-8B-Instruct --exclude "original/*" --local-dir meta-llama/Meta-Llama-3-8B-Instruct

In [ ]:
llama_model_id = "/teamspace/studios/this_studio/meta-llama/Meta-Llama-3-8B-Instruct"
llama_tokenizer = AutoTokenizer.from_pretrained(llama_model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Token-Based Chunking
def tokenize_and_chunk(text, chunk_size=512, overlap=128):
    input_ids = llama_tokenizer.encode(text, truncation=False)
    chunks = []
    for i in range(0, len(input_ids), chunk_size - overlap):
        chunk_ids = input_ids[i:i + chunk_size]
        chunk_text = llama_tokenizer.decode(chunk_ids, skip_special_tokens=True)
        chunks.append(chunk_text)
    return chunks

In [ ]:
# Generate Chunks
chunked_texts = []
chunk_metadata = []

for doc_id, row in tqdm(df.iterrows(), total=len(df)):
    input_text = str(row["Input"]) if pd.notna(row["Input"]) else ""
    chunks = tokenize_and_chunk(input_text)
    for i, chunk in enumerate(chunks):
        chunked_texts.append(chunk)
        chunk_metadata.append({
            "doc_id": doc_id,
            "chunk_index": i,
            "title": row.get("Title", f"Case {doc_id}"),
            "original_text": input_text
        })

100%|██████████| 15222/15222 [08:36<00:00, 29.49it/s]


In [ ]:
# Load Precomputed Embeddings
chunk_embeddings = np.load("chunk_embeddings.npy")

# Load FAISS Index
index = faiss.read_index("faiss_index.index")

In [8]:
def get_chunk_cls_embedding(text):
    inputs = embed_tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = embed_model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :]
    return cls_embedding.squeeze(0).cpu().numpy()

In [ ]:
# DO NOT RUN AGAIN
# InLegalBERT Embedding (CLS Token per Chunk)
def get_chunk_cls_embedding(text):
    inputs = embed_tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = embed_model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :]
    return cls_embedding.squeeze(0).cpu().numpy()

chunk_embeddings = []
for chunk in tqdm(chunked_texts, desc="Embedding chunks"):
    chunk_embeddings.append(get_chunk_cls_embedding(chunk))

chunk_embeddings = np.vstack(chunk_embeddings).astype("float32")

Embedding chunks: 100%|██████████| 201254/201254 [47:14<00:00, 71.01it/s] 


In [ ]:
# DO NOT RUN AGAIN
# ========== 7. Build FAISS Index ==========
embedding_dim = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(chunk_embeddings)

In [ ]:
# DO NOT RUN AGAIN
faiss.write_index(index, "faiss_index.index")
index = faiss.read_index("faiss_index.index")

In [ ]:
# Retrieve Chunks
def retrieve_chunks(query, top_k=6):
    query_vec = get_chunk_cls_embedding(query).reshape(1, -1)
    _, indices = index.search(query_vec, top_k)
    results = []
    for i in indices[0]:
        results.append({
            "chunk": chunked_texts[i],
            "doc_id": chunk_metadata[i]["doc_id"],
            "title": chunk_metadata[i]["title"],
            "chunk_index": chunk_metadata[i]["chunk_index"],
            "original_text": chunk_metadata[i]["original_text"]
        })
    return results

In [18]:
def truncate_chunks_to_fit_prompt(query_text, retrieved_chunks, max_tokens=4000, reserved_tokens=512):
    static_prompt = "You are a legal assistant AI trained to analyze legal documents.\n\nContext:\n"
    token_budget = max_tokens - len(llama_tokenizer.encode(static_prompt + query_text)) - reserved_tokens

    selected_chunks = []
    total_tokens = 0
    excluded_chunks = []

    for c in retrieved_chunks:
        tokens = llama_tokenizer.encode(c["chunk"])
        if total_tokens + len(tokens) > token_budget:
            excluded_chunks.append(c)
            continue
        selected_chunks.append(c)
        total_tokens += len(tokens)

    return selected_chunks, excluded_chunks


In [ ]:
# Build RAG Prompt
def create_rag_prompt(query_text, retrieved_chunks):
    context = "\n\n".join([f"Chunk {i+1} (from {c['title']}):\n{c['chunk']}" for i, c in enumerate(retrieved_chunks)])
    return (
        f"You are a legal assistant AI trained to analyze legal documents.\n\n"
        f"Context:\n{context}\n\n"
        f"New Case:\n{query_text}\n\n"
        f"Task:\n"
        f"1. Predict whether the appeal will be accepted (1) or rejected (0).\n"
        f"2. Identify the most relevant sentence(s) from the chunks.\n"
        f"3. Explain your reasoning briefly (max 2 lines).\n\n"
        f"Output format:\n"
        f"Label: <0 or 1>\n"
        f"Explanation: <brief explanation>"
    )

In [ ]:
import pandas as pd
from tqdm import tqdm
import os
import csv

OUTPUT_CSV_PATH = "prompts_rag_LLM.csv"

# Prepare a list of inputs (queries) and metadata from df
records = []

print("Generating prompts from truncated chunks...")

for doc_id, row in tqdm(df.iterrows(), total=len(df)):
    query_text = str(row["Input"]) if pd.notna(row["Input"]) else ""
    if not query_text.strip():
        continue

    # Retrieve top-k chunks
    retrieved = retrieve_chunks(query_text, top_k=6)

    # Truncate to fit token budget
    selected_chunks, excluded_chunks = truncate_chunks_to_fit_prompt(query_text, retrieved)

    # Build prompt
    prompt = create_rag_prompt(query_text, selected_chunks)

    # Create record
    records.append({
        "doc_id": doc_id,
        "title": row.get("Title", f"Case {doc_id}"),
        "query": query_text,
        "num_retrieved_chunks": len(retrieved),
        "num_selected_chunks": len(selected_chunks),
        "excluded_chunks": "; ".join([c["chunk"][:100] for c in excluded_chunks]),  # optional preview
        "prompt": prompt,
        "label": row.get("Label", "")
    })

# Save to CSV
df_prompts = pd.DataFrame(records)
df_prompts.to_csv(OUTPUT_CSV_PATH, index=False, quoting=csv.QUOTE_NONNUMERIC)

print(f"Saved {len(df_prompts)} prompts to {OUTPUT_CSV_PATH}")

Generating prompts from truncated chunks...


100%|██████████| 15222/15222 [18:43<00:00, 13.54it/s]


Saved 14979 prompts to prompts_rag_LLM.csv


In [2]:
import pandas as pd

df_prompts = pd.read_csv('/Users/gunjananand/Desktop/Lightning AI-Palak/prompts_rag_LLM.csv')

In [3]:
df_prompts

,doc_id,title,query,num_retrieved_chunks,num_selected_chunks,excluded_chunks,prompt,label
0,0,Case 0,"S. ABDUL NAZEER, J. 1. In this appeal, appella...",6,3,"R. BANUMATHI, J. 1. Leave granted. 2. This app...",You are a legal assistant AI trained to analyz...,0
1,1,Case 1,"SANJIV KHANNA, J. 1. Leave granted. 2. Kanwar ...",6,0,"Rastogi, J. 1. The instant appeal is directed ...",You are a legal assistant AI trained to analyz...,1
2,2,Case 2,"Santosh Hegde, J. 1. The incident which gives ...",6,3,Dial Singh started loading his pistol he fire...,You are a legal assistant AI trained to analyz...,1
3,3,Case 3,"S.K. Das, J. 1. These two appeals and the conn...",6,1,"Grover, J.1. This is an appeal from a judgment...",You are a legal assistant AI trained to analyz...,0
4,4,Case 4,"Shah, J. 1. Two persons-Walter and Ramasubramo...",6,0,"Alagiriswami, J.1. This is an appeal by specia...",You are a legal assistant AI trained to analyz...,1
...,...,...,...,...,...,...,...,...
14974,3039,Case 3039,"OJHA, J. 1. These appeals raise an identical q...",6,0,"the aforesaid bedrock, the appellate authorit...",You are a legal assistant AI trained to analyz...,0
14975,3040,Case 3040,"A.K. PATNAIK, J. 1. Leave granted. 2. This is ...",6,0,"K. JAYACHANDRA REDDY, J. Leave granted 2. Both...",You are a legal assistant AI trained to analyz...,1
14976,3041,Case 3041,"Sikri, J. 1. This appeal by certificate grante...",6,2,"Subba Rao, J.1. This appeal by special leave i...",You are a legal assistant AI trained to analyz...,0
14977,3042,Case 3042,"R.K. Agrawal, J.1. The above appeal has been f...",6,0,"R.K. Agrawal, J.1. The above appeal has been f...",You are a legal assistant AI trained to analyz...,0


In [4]:
df_prompts['prompt'][14974]

'You are a legal assistant AI trained to analyze legal documents.\n\nContext:\n\n\nNew Case:\nOJHA, J. 1. These appeals raise an identical question and are as such being decided by a common judgment. Before coming to the question involved in these appeals it would be necessary to give in brief the facts of each of these cases to indicate the circumstances in which the said question arises. Civil Appeal Nos. 4291-4292 of 1984 have been preferred against the judgment dated 18th April, 1984 of the Madras High Court in Writ Appeal Nos. 561 and 562 of 1983. C.V. Raman, the appellant in these two appeals was an employee in the Bank of India which is a Nationalised Bank. He was dismissed from service in pursuance of disciplinary action for certain charges framed against him. Aggrieved, he preferred an appeal under Section 41(2) of the Tamil Nadu Shops and Establishments Act, 1947 (hereinafter referred to as the Tamil Nadu Shops Act). A preliminary objection was raised on behalf of the Bank of

In [4]:
GROQ_MODEL = "llama-3.3-70b-versatile"
GROQ_API_URL = "https://api.groq.com/openai/v1/chat/completions"
OUTPUT_CSV_PATH = "InLegalBERT_llama_Groq_Output.csv"
INPUT_CSV_PATH = "prompts_rag_LLM.csv"

In [ ]:
import requests
import time
import random
import pandas as pd
from tqdm import tqdm
import os
import csv

# Groq API keys
GROQ_API_KEYS = [
    "groq_key1",
    "groq_key2",
    "groq_key3"
]

# Load prompts and optionally resume 
df_prompts = pd.read_csv(INPUT_CSV_PATH)
completed_ids = set()

if os.path.exists(OUTPUT_CSV_PATH):
    df_existing = pd.read_csv(OUTPUT_CSV_PATH)
    completed_ids = set(df_existing["doc_id"])

# Groq request function with retry and key rotation 
def generate_with_groq(prompt, max_retries=3, backoff_base=2, max_tokens=512):
    for attempt in range(max_retries * len(GROQ_API_KEYS)):
        key_index = attempt % len(GROQ_API_KEYS)
        api_key = GROQ_API_KEYS[key_index]
        wait_time = backoff_base ** (attempt // len(GROQ_API_KEYS))

        headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }

        payload = {
            "model": GROQ_MODEL,
            "messages": [
                {"role": "system", "content": "You are a helpful legal assistant."},
                {"role": "user", "content": prompt}
            ],
            "temperature": 0.3,
            "top_p": 0.9,
            "max_tokens": max_tokens
        }

        try:
            response = requests.post(GROQ_API_URL, headers=headers, json=payload)
            if response.status_code == 200:
                return response.json()["choices"][0]["message"]["content"].strip()
            else:
                print(f"[Key {key_index}] Error {response.status_code}: {response.text}")
        except Exception as e:
            print(f"[Key {key_index}] Exception: {str(e)}")

        print(f"[Key {key_index}] Retrying in {wait_time}s...")
        time.sleep(wait_time)

    return "[ERROR] All keys failed after retries"

# Run inference and write outputs incrementally
with open(OUTPUT_CSV_PATH, "a", newline="", encoding="utf-8") as f_out:
    writer = csv.DictWriter(f_out, fieldnames=["doc_id", "title", "label", "prompt", "model_output"])
    if os.stat(OUTPUT_CSV_PATH).st_size == 0:
        writer.writeheader()

    for i, row in tqdm(df_prompts.iterrows(), total=len(df_prompts)):
        if row["doc_id"] in completed_ids:
            continue

        prompt = row["prompt"]
        model_output = generate_with_groq(prompt)

        writer.writerow({
            "doc_id": row["doc_id"],
            "title": row["title"],
            "label": row["label"],
            "prompt": prompt,
            "model_output": model_output
        })
        f_out.flush()


 50%|█████     | 7554/14979 [00:00<00:00, 43121.77it/s]

[Key 0] Error 429: {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jxepatw9fkcr5bgtfvp5dax5` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 98246, Requested 3911. Please try again in 31m3.287s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}

[Key 0] Retrying in 1s...
[Key 1] Error 429: {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01js18a1zdfhb8hsngkj5xx7me` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 97799, Requested 3911. Please try again in 24m37.032s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}

[Key 1] Retrying in 1s...
[Key 0] Error 429: {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jxepatw9fkcr5bgtfvp

 66%|██████▌   | 9837/14979 [00:15<00:10, 489.40it/s]  

[Key 0] Error 429: {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jxepatw9fkcr5bgtfvp5dax5` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 98228, Requested 3327. Please try again in 22m23.428999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}

[Key 0] Retrying in 1s...
[Key 1] Error 429: {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01js18a1zdfhb8hsngkj5xx7me` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 97781, Requested 3327. Please try again in 15m57.173s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}

[Key 1] Retrying in 1s...
[Key 0] Error 429: {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jxepatw9fkcr

 66%|██████▌   | 9837/14979 [00:35<00:10, 489.40it/s]

...

 81%|████████▏ | 12176/14979 [2:11:18<2:30:08,  3.21s/it]

[Key 0] Error 429: {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jxepatw9fkcr5bgtfvp5dax5` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99088, Requested 4083. Please try again in 45m39.491s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}

[Key 0] Retrying in 1s...
[Key 1] Error 429: {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01js18a1zdfhb8hsngkj5xx7me` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 97306, Requested 4083. Please try again in 19m59.736s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}

[Key 1] Retrying in 1s...


 81%|████████▏ | 12177/14979 [2:11:21<2:26:38,  3.14s/it]

[Key 0] Error 429: {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jxepatw9fkcr5bgtfvp5dax5` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99085, Requested 3461. Please try again in 36m39.119s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}

[Key 0] Retrying in 1s...
[Key 1] Error 429: {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01js18a1zdfhb8hsngkj5xx7me` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 97303, Requested 3461. Please try again in 10m59.372999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}

[Key 1] Retrying in 1s...


100%|██████████| 14979/14979 [2:11:24<00:00,  1.90it/s]  


In [5]:
df_inference = pd.read_csv(OUTPUT_CSV_PATH)
df_inference.head()

,doc_id,title,label,prompt,model_output
0,0,Case 0,0,You are a legal assistant AI trained to analyz...,Label: 0\nExplanation: The appeal is rejected ...
1,1,Case 1,1,You are a legal assistant AI trained to analyz...,Label: 0\nExplanation: The court's reasoning a...
2,2,Case 2,1,You are a legal assistant AI trained to analyz...,Label: 1\nExplanation: The appeal will be acce...
3,3,Case 3,0,You are a legal assistant AI trained to analyz...,Label: 0\nExplanation: The court agrees with t...
4,4,Case 4,1,You are a legal assistant AI trained to analyz...,Label: 1\nExplanation: The court's reasoning s...


In [ ]:
import pandas as pd
import re
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Parse predicted labels from model output
def extract_label(text):
    if not isinstance(text, str):
        return "missing label"
    match = re.search(r"Label:\s*(\d)", text)
    if match:
        return int(match.group(1))
    return "missing label"

df_inference["predicted_label"] = df_inference["model_output"].apply(extract_label)

# Create evaluation subset
df_eval = df_inference[df_inference["predicted_label"] != "missing label"].copy()
df_eval["label"] = df_eval["label"].astype(int)
df_eval["predicted_label"] = df_eval["predicted_label"].astype(int)

# Calculate metrics
y_true = df_eval["label"]
y_pred = df_eval["predicted_label"]

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# Print evaluation results
print("Evaluation Results (Valid Predictions Only)")
print("------------------------------------------")
print(f"Evaluated rows       : {len(df_eval)} / {len(df_inference)} total")
print(f"Missing label rows   : {df_inference['predicted_label'].value_counts().get('missing label', 0)}")
print(f"Accuracy             : {accuracy:.4f}")
print(f"Precision (label=1)  : {precision:.4f}")
print(f"Recall    (label=1)  : {recall:.4f}")
print(f"F1 Score             : {f1:.4f}")
print("\nDetailed Classification Report:\n")
print(classification_report(y_true, y_pred, digits=4))


Evaluation Results (Valid Predictions Only)
------------------------------------------
Evaluated rows       : 12165 / 12178 total
Missing label rows   : 13
Accuracy             : 0.8790
Precision (label=1)  : 0.9021
Recall    (label=1)  : 0.8677
F1 Score             : 0.8846

Detailed Classification Report:

              precision    recall  f1-score   support

           0     0.8545    0.8919    0.8728      5664
           1     0.9021    0.8677    0.8846      6501

    accuracy                         0.8790     12165
   macro avg     0.8783    0.8798    0.8787     12165
weighted avg     0.8800    0.8790    0.8791     12165

